In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#constants
LANE_WIDTH = 340
challenge = 'challenge_video.mp4'
normal = 'project_video.mp4'
cap = cv2.VideoCapture("project_data\\"+ normal )
# debug = 1

In [2]:
def PerspectiveTransform(src,dst):
    mat = cv2.getPerspectiveTransform(src,dst)
    mat_inv = cv2.getPerspectiveTransform(dst,src)
    return mat,mat_inv

def warpPerspective(img, mat, size):
    return cv2.warpPerspective(img, mat,size, cv2.INTER_LINEAR)

#function for converting the image to hls mode then return s as it is the one we are intrested in studying
def trans2hls(image):
    img_hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    (h,l,s) = cv2.split(img_hls)
    #MODIFIED
    return (l,s)

In [3]:
#images must have the same dimension
#takes a list of images and produces an image of them concantenated in a 2D array-like shape
#need to take care of colored images 
def debug_mode(images , resize_factor):
    x_dim = int(images[0].shape[1] / resize_factor)
    y_dim = int(images[0].shape[0] / resize_factor)
    n_h_images = len(images)
    
    #if there are odd number of images add a dummy black image to the end of the list
    if (n_h_images % 2 )!= 0:
        images.append(np.zeros_like(images[n_h_images - 1]))    
        
    i = 0
    horiz_conc_image = list()
    while i < n_h_images:
        #if gray scale convert to RGB
        if len(images[i].shape) == 2:
            images[i] = cv2.resize(cv2.cvtColor(images[i],cv2.COLOR_GRAY2BGR), (x_dim , y_dim), interpolation= cv2.INTER_LINEAR)
        else:
            images[i] = cv2.resize(images[i], (x_dim , y_dim), interpolation= cv2.INTER_LINEAR)
        if len(images[i+1].shape) == 2:
            images[i + 1] = cv2.resize(cv2.cvtColor(images[i+1],cv2.COLOR_GRAY2BGR), (x_dim , y_dim), interpolation= cv2.INTER_LINEAR)
        else:
            images[i + 1] = cv2.resize(images[i+1], (x_dim , y_dim), interpolation= cv2.INTER_LINEAR)
        #concatenate each two successive images horizontally
        horiz_conc_image.append(cv2.hconcat([images[i] , images[i+1]]))
        i += 2
    #concatenate the resulting horizontally conc. images vertically
    debug_image = cv2.vconcat(horiz_conc_image)

    return debug_image

In [4]:
def detect_edges(image , s_thresh , l_thresh , shad_thresh= (50,155)):
    hls_image = color_to_hls(image)
    (h,l,s) = cv.split(hls_image)
    
    canny_l = cv.Canny(l,l_thresh[0],l_thresh[1])
    ext_shadow = np.zeros_like(h)
    ext_shadow[(l < shad_thresh[0]) & (s > shad_thresh[1])] = 1
    
    
#     canny_l[canny_l == 255] = 1
    s_binary = np.zeros_like(s)
    s_binary[(s >= s_thresh[0]) & (s <= s_thresh[1])] = 255
    
    combined_binary = np.zeros_like(canny_l)
    combined_binary[(s_binary == 255) | (canny_l == 255)] = 255
    combined_binary[ext_shadow == 1] = 0
    return combined_binary,s_binary,cany_l

In [5]:
def sliding_window(img,dst_colored, window_size , right_poly_old, init , frame_discard):
    #window_size = (hight , width)
    #shape= (720 * 1280)
    #convert to colored img to draw colored line and windows on top of it
    
    out_img = cv.cvtColor(img , cv.COLOR_GRAY2RGB)
    
    nwindows = int(img.shape[0] / window_size[0])
   
    # find peaks of left and right lanes
    histogram = np.sum(img, axis=0)
    midpoint = int(histogram.shape[0]//2)
    start_left_x= np.argmax(histogram[:midpoint - 115])
    start_right_x = np.argmax(histogram[midpoint + 100:]) + midpoint + 100
    
    #get positions of white pixels in original img
    white_pixels = img.nonzero()
    white_x = np.array(white_pixels[1])
    white_y = np.array(white_pixels[0])

    
    # the left and right lane indices that we are going to find
    left_lane_indices = []
    right_lane_indices = []
    
    for window in range(nwindows):
        
        # find the boundary of each window
        win_bot = img.shape[0] - (window+1)*window_size[0]
        win_top = img.shape[0] - window*window_size[0]
        left_lane_lbound = start_left_x - window_size[1]
        left_lane_rbound = start_left_x + window_size[1]
        right_lane_lbound = start_right_x - window_size[1]
        right_lane_rbound = start_right_x + window_size[1]
        
        #draw the windows in red
        cv.rectangle(dst_colored,(left_lane_lbound,win_bot),(left_lane_rbound,win_top),(255,0,0), 3) 
        cv.rectangle(dst_colored,(right_lane_lbound,win_bot),(right_lane_rbound,win_top),(255,0,0), 3) 
        
        #locate the white pixels that lie within current window 
        good_left_inds = ((white_y >= win_bot) & (white_y < win_top) & 
        (white_x >= left_lane_lbound) &  (white_x < left_lane_rbound)).nonzero()[0]
        good_right_inds = ((white_y >= win_bot) & (white_y < win_top) & 
        (white_x >= right_lane_lbound) &  (white_x < right_lane_rbound)).nonzero()[0]
        
        left_lane_indices.append(good_left_inds)
        right_lane_indices.append(good_right_inds)
        
        #if the window contain black pixels don't shit it
        if len(good_left_inds) > 65:
            start_left_x = int(np.mean(white_x[good_left_inds]))
        if len(good_right_inds) > 65:        
            start_right_x = int(np.mean(white_x[good_right_inds]))

            
    left_lane_indices = np.concatenate(left_lane_indices)
    right_lane_indices = np.concatenate(right_lane_indices)

    
    leftx = white_x[left_lane_indices]
    lefty = white_y[left_lane_indices] 
    rightx = white_x[right_lane_indices]
    righty = white_y[right_lane_indices] 

    #fit a 2nd degree curve to the white pixels positions we found
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    
    #predict the current position
    ploty = np.linspace(0, 719, 720 )
    left_fitx = np.polyval(left_fit , ploty)
    right_fitx = np.polyval(right_fit , ploty)
    
    left_poly = np.asarray(tuple(zip(left_fitx,ploty)) ,np.int32)
    right_poly = np.asarray(tuple(zip(right_fitx,ploty)),np.int32)
    if init == True:
        if abs(right_poly_old[30][0] - right_poly[30][0]) > frame_discard:
            right_poly = right_poly_old
    
    #draw the lanes 
    cv.polylines(dst_colored , [left_poly] , isClosed = False , color=(0,255,0) , thickness=50)
    cv.polylines(dst_colored , [right_poly], isClosed = False , color=(0,0,255), thickness= 50)
    
    return dst_colored, right_poly,left_fitx , right_fitx,ploty

In [6]:
#defining points for perspective transform
input_top_left = [580,480]
input_top_right = [820,480]
input_bottom_right = [1200,700]
input_bottom_left = [200,700]
src_points = np.float32([input_bottom_left,input_top_left,input_top_right,input_bottom_right])


In [7]:
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        continue
    #code for modifying video frames
    dst_points = np.float32([[frame.shape[1] // 3,720],[frame.shape[1] // 3,0],[2 *( frame.shape[1] // 3),0],[2 * (frame.shape[1] // 3),720]])
    frame = detectLane(frame, src_points, dst_points, (1280,720),kernel_op_size=2 , kernel_cl_size=7 , debug=1 )
    cv2.imshow('window-name', frame)
    
    count = count + 1
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()